In [2]:
import datetime
import logging
import pandas as pd 
import os
from google.cloud import bigquery 

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/krishna/Downloads/My Project 11220-3961afce01cc.json"

In [4]:
def get_bq_data(projects):
    """Gets metadata from public BigQuery datasets"""
    data = []
    for project in projects:
        client = bigquery.Client(project=project)
        datasets = client.list_datasets()
 
        for dataset in datasets:
            dataset_id = dataset.dataset_id
            dataset_ref = client.get_dataset(dataset_id)
            tables = client.list_tables(dataset_id)
            for table in tables:
                full_table_id = table.full_table_id.replace(':', '.')
                table_ref = client.get_table(full_table_id)
                item = {'dataset_id': dataset_id,
                        'project_id': project,
                        'table_id': table_ref.table_id,
                        'dataset_description': dataset_ref.description,
                        'table_modified': table_ref.modified.strftime("%Y-%m-%d %H:%M:%S"),
                        'table_created': table_ref.created.strftime("%Y-%m-%d %H:%M:%S"),
                        'table_description': table_ref.description,
                        'table_num_bytes': table_ref.num_bytes,
                        'table_num_rows': table_ref.num_rows,
                        'table_partitioning_type': table_ref.partitioning_type,
                        'table_type': table_ref.table_type,
                        'table_columns' : field_details(table_ref),
                        
                        }
                data.append(item)
    return data

In [5]:
def field_details(table_ref):
    schema_dict = table_ref.to_api_repr()["schema"]["fields"]
    fields = [_["name"] for _ in schema_dict]
    return fields

In [1]:

projects = [
    'bigquery-public-data',
]

In [100]:
big_query_data = get_bq_data(projects)

In [102]:
df =pd.DataFrame(res)

In [103]:
df.head(2)

,dataset_description,dataset_id,project_id,table_columns,table_created,table_description,table_id,table_modified,table_num_bytes,table_num_rows,table_partitioning_type,table_type
0,None,austin_311,bigquery-public-data,"[unique_key, complaint_type, complaint_descrip...",2017-05-18 14:29:56,None,311_service_requests,2019-01-05 00:58:32,185335640,640525,None,TABLE
1,None,austin_bikeshare,bigquery-public-data,"[station_id, name, status, latitude, longitude...",2017-05-25 03:43:34,None,bikeshare_stations,2019-01-24 10:44:40,7384,94,None,TABLE


In [104]:
df.to_csv("/home/krishna/projects/lineage/data/public_dataset_metadata_fin.csv")

In [6]:
fh_query_data = get_bq_data(['fh-bigquery'])

In [7]:
fh_df = pd.DataFrame(fh_query_data)

In [8]:
fh_df.shape

(1164, 12)

In [9]:
fh_df.head()

,dataset_description,dataset_id,project_id,table_columns,table_created,table_description,table_id,table_modified,table_num_bytes,table_num_rows,table_partitioning_type,table_type
0,None,bigdataspain,fh-bigquery,"[transaction, frm_amount, to_amount, time, blo...",2013-11-07 18:50:43,None,aggr_transaction_clean,2013-11-07 18:50:43,630730636,3846825,None,TABLE
1,None,bigdataspain,fh-bigquery,"[transaction, frm_amount, to_amount, time, blo...",2013-11-07 17:58:36,None,aggr_transactions,2013-11-07 18:04:12,696837724,4249917,None,TABLE
2,None,bigdataspain,fh-bigquery,"[transaction, frm_amount, to_amount, time, blo...",2013-11-07 18:13:35,None,aggr_transactions2,2013-11-07 18:44:05,696837724,4249917,None,TABLE
3,None,bigdataspain,fh-bigquery,"[transaction, frm_amount, time, block, hsh]",2013-11-07 17:34:09,None,aggr_transactions_frm,2013-11-07 18:03:34,662849836,4249917,None,TABLE
4,None,bigdataspain,fh-bigquery,"[transaction, frm_amount, time, block, hsh]",2013-11-07 18:37:44,None,aggr_transactions_frm2,2013-11-07 18:37:44,662849836,4249917,None,TABLE


In [10]:
fh_df.to_csv("/home/krishna/projects/lineage/data/fh_big_query_data.csv",index=False)